# Sreality - Scraping
## Jirka Zelenka
### 12.3.-24.4.2020
### Project = Scraping + Cleaning & Dropping + Visualization + All_In_One + PowerBI
-----------------------------------------------------------------------------

### Prerequisities:
* chromedriver.exe // edgedriver.exe
* pre-isntalled packages
* access to the file "Adresy.xlsx"

### Content:
* 1) Importing packages
* 2) Scraping URLs
* 3) JSON - coordinates, decsription, price
* 4) Getting the rest of info
* 5) Mapping adresses - previous offers + Nominatim GeoPy
* 6) Run
* 7) BONUS - Music part

-----------------------
## 1) Importing packages
-----------------------

In [101]:
#  1 = Scraping ###############################################################################################################################

##### General ############
import pandas as pd                     
import numpy as np                      
import matplotlib as plt                
from matplotlib.pyplot import figure    

from tqdm import tqdm  
import os
from collections import Counter         
from datetime import datetime           
import re                               
from time import sleep                 
import random                           
import math                             
import time                             
import itertools       
import sys  

##### Scraping ############
import requests                        
from bs4 import BeautifulSoup          
from selenium import webdriver         
import json                           

##### GeoPy ############        
from geopy.geocoders import Nominatim   
from geopy.exc import GeocoderTimedOut  

##### Widgets ############
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
from IPython.display import Image

##### Bonus - Music ############
import winsound                   

##### Visualization ############
import seaborn as sns               


-----------------------
## 2) Scraping URLs
-----------------------

In [2]:
print(requests.get("https://www.sreality.cz/robots.txt").text)

User-agent: *
Disallow: /

User-agent: Googlebot
Allow: /
Disallow: /advertpdf/
Disallow: /favourites-info
Disallow: *_buri=
Disallow: /adresar/*page=
Disallow: /adresar/*perPage=
Disallow: /adresar/*search=
Disallow: /adresar/*letter=
Disallow: /adresar/*id=
Disallow: /firma/*page=
Disallow: /firma/*perPage=
Disallow: /firma/*search=
Disallow: /firma/*letter=
Disallow: /firma/*id=
Disallow: /hledani/*,
Allow: /hledani/*region=*,
Disallow: /rk-detail
Disallow: *bez-aukce=
Disallow: *without-auction=
Disallow: *pois_in_place=
Disallow: *pois_in_place_distance=
# extensive crawling of language versions
Disallow: /en/*/*/*/*/*/
Disallow: /en/*?*&*&*&*&
Disallow: /ru/*/*/*/*/*/
Disallow: /ru/*?*&*&*&*&
# extensive crawling of rea directory
Disallow: /adresar/*/*/inzeraty/*?*
Disallow: /en/directory/*/*/estates/*?*
Disallow: /ru/directory/*/*/estates/*?*
# extensive crawling of rea directory locations
Disallow: /adresar/*/*/inzeraty/*,*
Disallow: /en/directory/*/*/estates/*,*
Disallow: /ru/

In [104]:

# 1 = Scraping 

def get_soup(url):
    browser = webdriver.Edge(executable_path=f'{os.getcwd()}/msedgedriver.exe')
    browser.get(url)
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML, 'html.parser')
    browser.quit()
    return soup

def extract_links(soup, type_of_deal):
    elements = []
    for link in soup.findAll('a', attrs={'href': re.compile(f"^/detail/{type_of_deal}/")}):
        link = link.get('href')
        elements.append(link)
    return list(set(elements[0::2]))

def get_number_of_pages(soup):
    records = soup.find_all(class_='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))
    records = ",".join(records).replace(",", "")
    max_page = math.ceil(int(records) / 20)
    return records, max_page

def scrape_pages(url, pages, type_of_deal):
    elements = []

    #sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.") 
    for i in tqdm(range(2, pages+1)):
        url2 = f"{url}?strana={i}"
        soup2 = get_soup(url2)
        elements.extend(extract_links(soup2, type_of_deal))
        sleep(random.uniform(1.0, 1.5))
    return elements

def get_soup_elements(type_of_deal="prodej", type_of_building="byty", pages=1):
    
    base_url = "https://www.sreality.cz/hledani"
    url = f"{base_url}/{type_of_deal}/{type_of_building}"
    
    soup = get_soup(url)
    elements = extract_links(soup, type_of_deal)
    records, max_page = get_number_of_pages(soup)

    print("----------------")
    print(f"I am scraping: {type_of_deal} {type_of_building}")
    print(f"Total number of records: {records}")
    print(f"Total pages: {max_page}")

    if pages >= max_page:
        pages = max_page
        print(f"I am scraping all {pages} pages.")
    else:
        print(f"I am scraping only {pages} pages out of {max_page}")
        print("----------------")

    elements += scrape_pages(url, pages, type_of_deal)
    
    return elements


#  2 = Getting URLS 

def elements_and_ids(x):
    
    df = pd.DataFrame({"url":x})

    def get_id(x):
        return x.split("/")[-1]

    df["url_id"] = df["url"].apply(get_id)
                                                                          
    return df


#### Test with direct request

In [115]:
import requests

url = "https://www.sreality.cz/hledani/prodej/byty?strana=2"

res = requests.get(url)
print(res.text)
print(res.json())
#data = json.loads(res.text)
#print(data)

<!DOCTYPE html>
<html lang="{{ html.lang }}" ng-app="sreality" ng-controller="MainCtrl">
	<head>
		<meta charset="utf-8" />
		<meta http-equiv="X-UA-Compatible" content="IE=edge" />
		<meta
			name="viewport"
			content="width=device-width,initial-scale=1,maximum-scale=1.0,minimal-ui"
		/>

		<!--- Nastaveni meta pres JS a ne pres Angular, aby byla nastavena default hodnota pro agenty co nezvladaji PhantomJS --->
		<title ng:bind-template="{{metaSeo.title}}">
			Sreality.cz â¢ reality a nemovitosti z celÃ© ÄR
		</title>
		<meta
			name="description"
			content="NejvÄtÅ¡Ã­ nabÃ­dka nemovitostÃ­ v ÄR. NabÃ­zÃ­me byty, domy, novostavby, nebytovÃ© prostory, pozemky a dalÅ¡Ã­ reality k prodeji i pronÃ¡jmu. Sreality.cz"
		/>
		<meta
			property="og:title"
			content="Sreality.cz â¢ reality a nemovitosti z celÃ© ÄR"
		/>
		<meta property="og:type" content="website" />
		<meta
			property="og:image"
			content="https://www.sreality.cz/img/sreality-logo-og.png"
		/>
		<meta
			property="o

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

-----------------------
## 3) JSON - souřadnice, popis, cena
-----------------------

In [4]:
# 3 = získání Souřadnic, Ceny a Popisu = z JSON ################################################################################################################################  

def get_coords_price_meters(x):
    
    url = "https://www.sreality.cz/api/cs/v2/estates/" + str(x)
    
    response = requests.get(url)
    byt = json.loads(response.text, encoding = 'UTF-8')                                
    try:
        coords = (byt["map"]["lat"], byt["map"]["lon"])
    except:
        coords = (0.01, 0.01)  #Pro případ neexisutjících souřadnic
    try:
        price = byt["price_czk"]["value_raw"] 
    except:
        price = -1
        
    try:
        description = byt["meta_description"]
    except:
        description = -1
    
    return coords, price, description


# Severní Šířka = latitude
# výchoDní / zápaDní Délka = longitude

def latitude(x):                   #Rozdělí souřadnice na LAT a LON
    x = str(x).split()[0][1:8]
    return x

def longitude(x):
    x = str(x).split()[1][0:7]
    return x


-----------------------
## 4) Získávání zbylých informací
-----------------------

In [5]:
#  4 = Prodej + Dům + Pokoje = z URL ###############################################################################################################################

def characteristics(x):
    x = x.split("/")
    buy_rent = x[2]
    home_house = x[3]
    rooms = x[4]
    
    return buy_rent, home_house, rooms

#  5 =  Plocha z Popisu ###############################################################################################################################

# Upraveno pro čísla větší než 1000 aby je to vzalo
# Zároveň se to vyhne velikost "Dispozice", "Atpyický", atd.

def plocha(x):
    try:
        metry = re.search(r'\s[12]\s\d{3}\s[m]', x)[0] # SPecificky popsáno: Začíná to mezerou, pak 1 nebo 2, pak mezera, pak tři čísla, mezera a pak "m"
        metry = metry.split()[0] + metry.split()[1]     # Separuju Jedničku + stovky metrů, bez "m"
    except:
        try:
            metry = re.search(r'\s\d{2,3}\s[m]', x)[0]  #Mezera, pak 1-3 čísla, mezera a metr
            metry = metry.split()[0]                    # Separuju čísla, bez "m"
        except:
            metry = -1
    return metry

-----------------------
## 5) Mapping adres - předešlé inzeráty + Nominatim GeoPy
-----------------------

### 1) Státní správa
    https://www.statnisprava.cz/rstsp/ciselniky.nsf/i/CZ0201
    - problémy s abecedou, Brno-město nemá obce atd
### 2) Wiki
    https://cs.wikipedia.org/wiki/Seznam_katastr%C3%A1ln%C3%ADch_%C3%BAzem%C3%AD_v_okrese_Bene%C5%A1ov
    - taky celý dost na houby, nezačal jsem
### 3) Volby.cz
    https://www.volby.cz/pls/kv2018/kv31?xjazyk=CZ&xid=1
    - Bylo by hezké, jsou to tables, easy to scrapp, jen jiný počet než v 1) (75 vs 77 okresů)
### 4) Staťák - excel
    https://www.czso.cz/csu/czso/pocet-obyvatel-v-obcich-za0wri436p?fbclid=IwAR1haSspuynZB8Awn08WDriMkUcsUCz4fHH9Pw2CwMDVGHPGJERxaqbrVg8
    - Tohle asi bude top, Zuzka mě zachránila


* 14 krajů
* 77 okresů podle státní správy včetně PRAHA
* 6258 obcí a újezdů  K 27. květnu 2016 (ČSÚ)  


In [6]:
# 6 = Adresy z předešlých inzerátů a short_coords ###############################################################################################################################

# Vytvoření ořezaných souřadnic, přesnost je dostatečná, lépe se najdou duplikáty
def short_coords(x):
    """
    x = x.astype(str)   # Bylo potřeba udělat string - ale Tuple se blbě převádí - vyřešil jsem uložením a načtením skrz excel
    """
    
    x1 = re.split(r'\W+', x)[1] + "."+re.split(r'\W+', x)[2]
    x1 = round(float(x1), 4)

    x2 = re.split(r'\W+', x)[3] + "."+re.split(r'\W+', x)[4]
    x2 = round(float(x2), 4)

    return (x1, x2)

#############################

# Napmapuje až 80 % Adres z předešlých inzerátů
def adress_old(x):  

    adresy = pd.read_excel("Adresy.xlsx")
    adresy = adresy[["oblast", "město", "okres", "kraj", "url_id", "short_coords"]]
    
    #Nejlepší napárování je toto:
    # alternativně Inner a Left minus řádky s NaNs a funguje stejně)
    
    x.short_coords = x.short_coords.astype(str)                              # získat string na souřadnice, protože v Načteném adresáři je mám už taky jako string
    data = pd.merge(x, adresy, on=["short_coords", "url_id"], how = "left")  #upraveno matchování na url_ID + short_coords, je to tak iv Adresáří, je to jednoznačné, jsou tam unikátní. 
                                                                            # Pokud si v dalším kroku dostáhnu ke starému url_id a k nové coords ještě novou adresu, tak pak se mi uloží do Adresáře nová kombiance ID + short_coord a je to OK
                                                                             # Viz funkce"update_databáze_adres() kde je totéž info
            
    print("-- Počet doplněných řádků je: " + str(len(data[~data.kraj.isna()])) + ", počet chybějících řádků je: "   + str(len(data[data.kraj.isna()])))
    
    return data

In [7]:
# 7 = Adresy - zbývající přes GeoLocator ###############################################################################################################################

def adress_new(x):

# Pozn. - je to random, závislost rychlosti na user_agent, i na format_string se nepovedlo potvrdit - ale dokumentace user-agent uvíádí jako povinnost
# Timeout na 20s  zrušil Errory - None záleží na verzi geopy !! viz dokumentace
#Rychlost a úspěch velmi záleží na připojení. Ryhlost 0.2s - 10s na záznam.
# Problém s Too many requests se "spraví přes noc", kdyžtak - nebo viz stackoverflow - nastavit user-agent (https://stackoverflow.com/questions/22786068/how-to-avoid-http-error-429-too-many-requests-python)
  
    geolocator = Nominatim(timeout = 20, user_agent = "JZ_Sreality")   # Pomohlo změnit jméno, proti "Error 403" !!        
    location = geolocator.reverse(x.strip("())"))   
                                                    # Reverse samotné znamená obrácené vyhleádvání = souřadnice -> Adresa
    try:
        oblast  = location[0].split(",")[-7]
    except:
        oblast  = -1
    try:
        město = location[0].split(",")[-6]
    except:
        město  = -1
    try:
        okres = location[0].split(",")[-5]
    except:
        okres  = -1
    try:
        kraj = location[0].split(",")[-4]  
    except:
        kraj  = -1       
    
    time.sleep(0.5)
    return oblast, město, okres, kraj

##################################################################

# Pomocná funkce, opakuje předchozí funkci pořád dokola dokud neprojde bez Erroru
def repeat_adress(x):
    try:
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))
    except GeocoderTimedOut:
        print("Another try")
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))


# 8 = Merging adres ###############################################################################################################################
# Aplikuje předchozí funkci pouze na řídky, které ještě nemají doplněné adresy z kroku 6.)

def adress_merging(x):

    data_new = x.copy()          
    bool_series = pd.isnull(data_new.kraj)                                   
    data_new = data_new[bool_series]     #subset s chybějícími adresami   
        
    repeat_adress(data_new)
        
    data_all = pd.concat([x, data_new], join_axes=[x.columns])   
    data_all = data_all[~pd.isnull(data_all.kraj)]
    data_all = data_all.sort_index()

    data = data_all.copy()
    
    return data


-----------------------
## 6) Spuštění
-----------------------

In [105]:

# Parametry:
# "prodej"/ "pronájem"
# "byty"/"domy"
# pages = 1- X, případně 999 = Všechny strany !

def scrap_all(type_of_deal = "prodej", type_of_building = "byty", pages = 1):
    
    # Scrapni data - hezky komunikuje = cca 50 min
    data = get_soup_elements(type_of_deal = type_of_deal,
                             type_of_building = type_of_building,
                             pages = pages)
    print( "1/8 Data scrapnuta, získávám URLs.")
    
    # 2 = Získání URLS
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...")
    """
    # 3 = získání Souřadnic, Ceny a Popisu = z JSON
    data["coords"], data["cena"], data["popis"] = zip(*data['url_id'].map(get_coords_price_meters))
    data["lat"] = data["coords"].apply(latitude)
    data["lon"] = data["coords"].apply(longitude)
    data.to_excel(r"a2_Souřadnice_prodej_byty.xlsx")
    print( "3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.")
   
    # 4 = Prodej + Dům + Pokoje = z URL
    data["prodej"], data["dům"],  data["pokoje"] = zip(*data['url'].map(characteristics))
    print("4/8 Získány informace z URLs, nyní získávám informace z popisu.")
    
    # 5 =  Plocha z Popisu
    data["plocha"] = data['popis'].apply(plocha)
    data.to_excel(r"a3_Popisky_prodej_byty.xlsx")
    print( "5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.")
    
   
    # 6 = Adresy z předešlých inzerátů a short_coords
    data = pd.read_excel(r"a3_Popisky_prodej_byty.xlsx")   # Abych se vyhnul konverzi TUPLE na STRING, což není triviální, tak si to radši uložím a znova načtu a získám stringy rovnou. Snad mi to nerozbije zbytek
    data["short_coords"] = data["coords"].apply(short_coords)
    data_upd = adress_old(data)                                # Tady nepotřebuji maping, protože se nesnažím něco nahodit na všechny řádky, ale merguju celé datasety
    data = data_upd.copy()
    print( "6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...")            # Přidat do printu počet řádků, kolik mám a kolik zbývá v 7. kroku

    # 7-8 = Adresy - zbývající přes GeoLocator + Merging

    try:                                    # !!! Riskuju že zas něco selže, jako USER- AGENT posledně...
        data_upd = adress_merging(data)    #Přidáno TRY pro situace, kdy už mám všechyn adresy z OLD a nejde nic namapovat !
        data = data_upd.copy()
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    
    except:
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 ŽÁDNÉ nové adresy. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    
    """
    
    return data

In [106]:
data = scrap_all(type_of_deal = "prodej", type_of_building = "byty", pages = 5)
print(len(data))

----------------
I am scraping: prodej byty
Total number of records: 17591
Total pages: 880
I am scraping only 5 pages out of 880
----------------


100%|██████████| 4/4 [00:39<00:00,  9.82s/it]


1/8 Data scrapnuta, získávám URLs.
-- Deleted 0 records because of deduplication.


ModuleNotFoundError: No module named 'openpyxl'

In [49]:
print(len(data))
data

100


['/detail/prodej/byt/4+1/praha-smichov-na-cihlarce/526894412',
 '/detail/prodej/byt/3+1/trebic-nove-dvory-c--boudy/409453900',
 '/detail/prodej/byt/3+1/brno-veveri-tuckova/1419105612',
 '/detail/prodej/byt/2+1/andelska-hora--/1859311436',
 '/detail/prodej/byt/5+kk/praha-stodulky-jindrova/761087052',
 '/detail/prodej/byt/2+1/ostrava-moravska-ostrava-28--rijna/1434789708',
 '/detail/prodej/byt/4+1/opava-predmesti-hradecka/1127273804',
 '/detail/prodej/byt/2+kk/svratouch--/4165727308',
 '/detail/prodej/byt/2+kk/praha--nurmiho/2040026956',
 '/detail/prodej/byt/3+kk/plzen-krimice-zitna/2288502092',
 '/detail/prodej/byt/3+1/chlumcany-chlumcany-v-rumunsku/3630679372',
 '/detail/prodej/byt/3+1/vlastiborice--/2036843852',
 '/detail/prodej/byt/3+kk/rostenice-zvonovice-rostenice-/1521427532',
 '/detail/prodej/byt/3+kk/rybnice-rybnice-/241681740',
 '/detail/prodej/byt/2+kk/rostenice-zvonovice-rostenice-/2676233292',
 '/detail/prodej/byt/1+kk/praha-lhotka-sturova/694666572',
 '/detail/prodej/byt/2+

### BONUS - We are the champions

In [3]:
import winsound
import time 

sec = 300  # milliseconds - for Beep
half_sec = 150
pause = 1   # Seconds - for Sleep

# Hz
A3 = 220
C4 = 262   #261.63
D4 = 294   #293.67
E4 = 330   #329.63
F4 = 350   #349.23
G4 = 392
A4 = 440
C5 = 523   #523.25


def we_are_the_champions(koef=2):
     
    winsound.Beep(koef*F4, 4*sec)
    winsound.Beep(koef*E4, sec)
    winsound.Beep(koef*F4, sec)
    winsound.Beep(koef*E4, 3*sec)
    winsound.Beep(koef*C4, 2*sec)
    winsound.Beep(koef*A3, 3*half_sec)
    winsound.Beep(koef*D4, 3*half_sec)
    winsound.Beep(koef*A3, 10*sec)
    time.sleep(pause)
    winsound.Beep(koef*C4, sec)
    winsound.Beep(koef*F4, 4*sec)
    winsound.Beep(koef*G4, 2*sec)
    winsound.Beep(koef*A4, 2*sec)
    winsound.Beep(koef*C5, 4*sec)
    winsound.Beep(koef*A4, 2*sec)
    winsound.Beep(koef*D4, sec)
    winsound.Beep(koef*E4, sec)
    winsound.Beep(koef*D4, 6*sec)

    
    
we_are_the_champions(2)